# Welcome to the Brazilian Apocalypse!

We are currently on the second year of the Coronavirus pandemic. It is obvious the institutions aren't ready to face extreme conditions for a prolonged period of time, and neither are the people. While there is a large community of paranoid people in the US who are ready to survive the extinction of civilization, in Brazil this scenario is not something we consider when thinking about the future.

Brazil is a country with continental dimensions, with more than 220 million people spread over 8.5 million square kilometers. There must be some dark forgotten places where one could ride out an extinction level event. There are more than five thousand cities and we will sort through different types of data to determine the best places to enjoy the end of the world.

After ranking the cities from best to worst, we will try to optimize our escape route, choosing our starting city and defining the sequence of cities to get to our final destination. 

## 1. Definition of the problem

There are several different causes for an apocalypse, and people are getting more creative everyday. Zombies, nuclear war, large asteroids, deadly viruses, global warming, exploding volcanoes, massive earthquakes, alien invasion! Although there are huge variations on each of these scenarios, we can look for some information that will help no matter what comes our way. 

Crowded places should be avoided, food must be plentiful, fresh water must be available. We also need medication to cure and prevent diseases, and firearms for protection. It would be smart to be away from nuclear power plants, but solar energy might be useful. Average temperatures around 22 degrees are a good choice. Being close to an airport shoud be interesting, even a small one, for a quick getaway, if you can find and fly a plane. 

We should also be worried about learning new abilities to survive and perhaps rebuild civilization, so we also looking at libraries and universities.

We will load data that helps to determine the potential for each city to fulfill one of this requirements, describing its source and how we create a metric to rank the cities. After processing all this information, our final data set will be a table where each row is a city and each column will show a score between 0 and 1 for a certain feature. Our last feature will be the sum of all scores, and we will rank the cities by this metric. Hopefully, we will have a clear winner for the whole country. 

We also have a list of neighboring cities for each location. So we can map a route from any place to the our sanctuary or try to find the best place at a certain distance from a starting point.

The common feature to all the data will be an ID given by the Brazilian Institute of Geography and Statistics, *IBGE*. This unique ID has six digits, but sometimes databases add an extra number at the end.

## 2. Extraction, Transformation and Loading of the Data

We are gathering here several different information for each Brazilian city. They were all gathered after a long, hard search through several government websites and other sources. We will present the source, the available info and the transformations needed to get our clean table.

All the libraries needed will be loaded here.

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import folium

### 2.1. 2013 Brazilian Human Development Atlas

The Human Development Index (HDI) is a statistic composite index of life expectancy, education, and per capita income indicators. In 2013, the latest report was released, showing the scores for each city, state and the country as a whole, for the years 1990, 2000 and 2010. It can be accessed at <a href="http://www.atlasbrasil.org.br/">Atlas Brasil</a>.

Each row of the table represents a city for one of the given years. There are some ID columns and then all the features that are used to compute the score, the Human Development Index, for each city. From this table, we will need three columns:
1. the city ID;
2. the city population in 2010;
3. the health portion of the HDI in 2010.

The ID will be used as a key to merge all the different data into a single table. The population will be used to compute the population density, and the Health HDI will be used as an indication of how well the city is capable of handle medical emergencies.

In [5]:
#loading the file, bringing the sheet with the information for the cities
df_atlas = pd.read_excel('Atlas 2013_municipal, estadual e Brasil.xlsx', sheet_name='MUN 91-00-10')
#filtering for 2010
df_atlas = df_atlas[df_atlas['ANO']==2010]
#keeping only the needed features
df_atlas_clean = df_atlas[['Codmun6','Codmun7','pesotot','IDHM_L']]
#the ID will be turned into a string
df_atlas_clean['Codmun6'] = df_atlas_clean['Codmun6'].astype(str)
df_atlas_clean['Codmun7'] = df_atlas_clean['Codmun7'].astype(str)

#renaming columns
df_atlas_clean.rename(columns = {'Codmun6':'CityID', 'Codmun7':'CityID_7', 'pesotot':'Population',
                              'IDHM_L':'Health_Index'}, inplace = True)
df_atlas_clean = df_atlas_clean.reset_index(drop=True)
df_atlas_clean.head()

<ipython-input-5-81a67aa98e03>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atlas_clean['Codmun6'] = df_atlas_clean['Codmun6'].astype(str)
<ipython-input-5-81a67aa98e03>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atlas_clean['Codmun7'] = df_atlas_clean['Codmun7'].astype(str)
/home/rodrigolink/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

,CityID,CityID_7,Population,Health_Index
0,110001,1100015,24392,0.763
1,110002,1100023,90353,0.806
2,110003,1100031,6313,0.757
3,110004,1100049,78574,0.821
4,110005,1100056,17029,0.799


In [6]:
len(df_atlas_clean)

5565

### 2.2. Territorial Data from IBGE - Brazilian Institute of Geography and Statistics

There are a lot of information available at the IBGE website, and at this time we will focus on the area of each of the cities, available at <a href="https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/15761-areas-dos-municipios.html">Áreas Territoriais</a>. This will allow us to compute the population density. We will also keep the name and the state of each city, because we will need to use that for data sets that don't have the cityID.

In [31]:
#loading the file, bringing the sheet with the information for the cities
df_area = pd.read_excel('AR_BR_RG_UF_RGINT_RGIM_MES_MIC_MUN_2020.xls', sheet_name='AR_BR_MUN_2020')
#keeping only the needed features
df_area_clean = df_area[['CD_GCMUN','AR_MUN_2020','NM_MUN_2020','NM_UF_SIGLA']]
#the ID will be turned into a string and removing last digit
df_area_clean['CD_GCMUN'] = df_area_clean['CD_GCMUN'].astype(str).str[:6]
#turning the name and state into a single column, inlowercase
df_area_clean['citystate'] = (df_area_clean['NM_MUN_2020']+'-'+df_area_clean['NM_UF_SIGLA']).str.lower()
#keeping only the needed features
df_area_clean = df_area_clean[['CD_GCMUN','NM_MUN_2020','NM_UF_SIGLA','AR_MUN_2020','citystate']]
#renaming columns
df_area_clean.rename(columns = {'CD_GCMUN':'CityID', 'AR_MUN_2020':'Area','NM_UF_SIGLA':'State','NM_MUN_2020':'City'}, 
                     inplace = True)
df_area_clean = df_area_clean.reset_index(drop=True)

df_area_clean.head()

<ipython-input-31-078cff7cab7a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_area_clean['CD_GCMUN'] = df_area_clean['CD_GCMUN'].astype(str).str[:6]
<ipython-input-31-078cff7cab7a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_area_clean['citystate'] = (df_area_clean['NM_MUN_2020']+'-'+df_area_clean['NM_UF_SIGLA']).str.lower()


,CityID,City,State,Area,citystate
0,110001,ALTA FLORESTA D'OESTE,RO,7067.127,alta floresta d'oeste-ro
1,110002,ARIQUEMES,RO,4426.571,ariquemes-ro
2,110003,CABIXI,RO,1314.352,cabixi-ro
3,110004,CACOAL,RO,3793.000,cacoal-ro
4,110005,CEREJEIRAS,RO,2783.300,cerejeiras-ro


In [8]:
len(df_area_clean)

5572

Here we see a problem. The number of cities is different in our data sets. It seems there are new cities since 2010.

### 2.3. Tabnet - DataSUS

There is a national database with information from the Ministry of Health, at <a href="http://tabnet.datasus.gov.br/cgi/deftohtm.exe?cnes/cnv/leiintbr.def">Tabnet</a>. From there, we will get the number of hospital beds available for each city, another indicator that will be taken into account for our medical needs.

In [9]:
#loading the file
df_hospital = pd.read_csv('leitoshospitalares.csv',sep=';')
#getting the city ID from ID+Name
df_hospital['Município'] = df_hospital['Município'].str[:6]
#renaming columns
df_hospital.rename(columns = {'Município':'CityID', 'Quantidade existente':'Hospital_beds'},
                              inplace = True)
df_hospital = df_hospital.reset_index(drop=True)
df_hospital.head()

,CityID,Hospital_beds
0,110001,49
1,110037,24
2,110040,16
3,110034,35
4,110002,240


In [10]:
len(df_hospital)

3549

Here we have another thing to be aware of. Not all cities have hospitals. We will have other features like this, and we will tackle this problem after loading all of them

### 2.4. National Energy Agency

From the National Energy Agency, ANEEL, we can get the data with the location and energy production of all the power plants <a href="https://www.aneel.gov.br/dados/geracao">here</a>. We will pivot this data to get the cities as rows and the kind of power plant as columns, with how much is produced as our values.

Here we will keep only four types of energy production: solar, wind, biomass and nuclear power. Solar power is easy to mantain, Biomass is a nice alternative, Wind turbines should work for a long time without maintenance, and Nuclear power should be as far away as possible, because it fill fail without the proper care. Hydreletric power plants are too complex to mantain, and thermoeletric are dependent of fuel.

In [11]:
#loading file
df_energy = pd.read_csv('EmpreendimentoOperacao.csv')
#keeping only the needed columns
df_energy = df_energy[['dscMunicipio','dscOrigemCombustivel','mdaPotenciaOutorgadakW']]
#removing power plants that don't have a location
df_energy = df_energy.dropna(axis = 0, how ='any')
#finding cityID in the name+ID
df_energy['pos'] = df_energy['dscMunicipio'].str.find('CodIbge:').astype(int)
#getting cityID
df_energy['CodMunic'] = df_energy.apply(lambda x: x['dscMunicipio'][x['pos']+9:x['pos']+16],axis=1)
#keeping only the needed columns
df_energy_clean = df_energy[['CodMunic','dscOrigemCombustivel','mdaPotenciaOutorgadakW']]
#making sure the power production is integer
df_energy_clean['mdaPotenciaOutorgadakW'] = df_energy_clean['mdaPotenciaOutorgadakW'].astype(int)
#grouping production by city and type
df_energy_final =df_energy_clean.groupby(['CodMunic','dscOrigemCombustivel'], as_index=False).agg({'mdaPotenciaOutorgadakW':sum})
#pivoting to get types as columns
df_energy_pivot = df_energy_final.pivot(index='CodMunic', columns='dscOrigemCombustivel', values='mdaPotenciaOutorgadakW')
#keeping only the needed columns
df_energy_pivot = df_energy_pivot.drop(columns=['Fóssil','Hídrica','Undi-Elétrica'])
#resetting index
df_energy_pivot.reset_index(inplace=True)

#renaming columns
df_energy_pivot.rename(columns = {'CodMunic':'CityID','Biomassa':'BiomassPower','Eólica': 'WindPower',
                                  'Solar':'SolarPower','Nuclear':'NuclearPower'},
                              inplace = True)

#filling empty values with zero
df_energy_pivot = df_energy_pivot.fillna(0)
df_energy_pivot['CityID'] = df_energy_pivot['CityID'].str[:6]
df_energy_pivot.head()

<ipython-input-11-f956d67404ac>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_energy_clean['mdaPotenciaOutorgadakW'] = df_energy_clean['mdaPotenciaOutorgadakW'].astype(int)


dscOrigemCombustivel,CityID,BiomassPower,WindPower,NuclearPower,SolarPower
0,110002,0.0,0.0,0.0,0.0
1,110006,0.0,0.0,0.0,0.0
2,110007,0.0,0.0,0.0,0.0
3,110008,0.0,0.0,0.0,0.0
4,110010,0.0,0.0,0.0,0.0


In [12]:
len(df_energy_pivot)

1625

Again, not all cities are present in this data set.

### 2.5. INPE - National Institute for Space Research

One of the labs that are a part of INPE is LABREN - Modeling and Study of Renewable Energy Resources Lab, that released the <a href="http://labren.ccst.inpe.br/atlas_2017.html">2017 Brazilian Solar Power Atlas</a>. As we said before, Solar Power is one of the best alternatives for our post-apocalyptic new life.

From this data set, we will get the average solar incidence for each city. One thing that will come in handy are the coordinates for the center of each city. This will be useful when we'll need the distance between the cities.

In [13]:
#loading the file
df_solar = pd.read_csv('direct_normal_means_sedes-munic.csv',sep=';')
#keeping only the needed features
df_solar_clean = df_solar[['LON','LAT','NAME','STATE','ANNUAL']]
#replacing the state name by the stateID
df_solar_clean = df_solar_clean.replace(['ACRE', 'ALAGOAS', 'AMAPÁ', 'AMAZONAS', 'BAHIA', 'CEARÁ',
       'DISTRITO FEDERAL', 'ESPÍRITO SANTO', 'GOIÁS', 'MARANHÃO',
       'MATO GROSSO', 'MATO GROSSO DO SUL', 'MINAS GERAIS', 'PARÁ',
       'PARAÍBA', 'PARANÁ', 'PERNAMBUCO', 'PIAUÍ', 'RIO DE JANEIRO',
       'RIO GRANDE DO NORTE', 'RIO GRANDE DO SUL', 'RONDÔNIA', 'RORAIMA',
       'SANTA CATARINA', 'SÃO PAULO', 'SERGIPE', 'TOCANTINS'],['AC', 'AL', 'AP', 'AM', 'BA', 'CE',
       'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 
        'RO', 'RR', 'SC', 'SP', 'SE', 'TO'])

#turning the name and state into a single column, in lowercase
df_solar_clean['citystate'] = (df_solar_clean['NAME']+'-'+df_solar_clean['STATE']).str.lower()
#getting the cityID from the area dataset
df_areasolar = pd.merge(df_solar_clean, df_area_clean, on='citystate', how='inner')
#keeping only the needed features
df_solar_final = df_areasolar[['CityID','LON','LAT','ANNUAL']]



#renaming columns
df_solar_final.rename(columns = {'LON':'Longitude','LAT':'Latitude','ANNUAL': 'SolarIncidence'},
                              inplace = True)

df_solar_final.head()

/home/rodrigolink/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,CityID,Longitude,Latitude,SolarIncidence
0,120010,-68.7463,-11.0109,3438
1,120025,-68.7442,-11.0293,3438
2,120005,-69.5658,-10.9394,3453
3,120070,-68.4973,-10.6520,3413
4,120017,-67.6768,-10.5749,3269


### 2.6. Forest-GIS

This website is a portal for geotechnical data, and it has a collection of links to interesting data sets. From <a href="https://forest-gis.com/download-gis-base-de-dados/">here</a>, we will get the climate information for each city:  the average temperature and the standard deviation during the year, the rainfall and the altitude.

In [14]:
#loading the file, bringing the sheet with the information for the cities
df_climate = pd.read_excel('KoppenBrazilianmunicipalities.xls', sheet_name='Data')
#computing necessary data
df_climate['MeanTemperature'] = df_climate[df_climate.columns[6:18]].mean(axis=1)
df_climate['StdDevTemperature'] = df_climate[df_climate.columns[6:18]].std(axis=1)
df_climate['MeanRainfall'] = df_climate[df_climate.columns[18:]].mean(axis=1)
#keeping only the needed features
df_climate_clean = df_climate[['IBGE-Code','Altitude', "MeanTemperature", "StdDevTemperature", 'MeanRainfall']]
#renaming columns
df_climate_clean.rename(columns = {'IBGE-Code':'CityID'},
                              inplace = True)


df_climate_clean['CityID'] = df_climate_clean['CityID'].astype(str).str[:6]
df_climate_clean.head()

/home/rodrigolink/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-14-8d5fa24ef143>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_climate_clean['CityID'] = df_climate_clean['CityID'].astype(str).str[:6]


,CityID,Altitude,MeanTemperature,StdDevTemperature,MeanRainfall
0,110001,246.084366,25.416845,1.537877,167.144504
1,110002,165.899734,25.934812,1.225573,177.045622
2,110003,224.183029,25.554969,1.655509,162.875902
3,110004,229.949814,25.830520,1.342803,171.251407
4,110005,192.215805,25.640308,1.666064,165.300759


In [15]:
len(df_climate_clean)

5567

### 2.7. SIDRA, IBGE's Automatic Retrieval System

From this platform we will get the tables with the agriculture production for each city. We have two inputs, <a href="https://sidra.ibge.gov.br/Tabela/1613">one</a> for the permanent farms and <a href="https://sidra.ibge.gov.br/Tabela/1612">another</a> for the temporary productions. The information we are seeking here is the area being used for plantations, and we will use that as a proxy to determine how fertile the region around the city is.

In [16]:
#loading files
df_agro1 = pd.read_excel('tabela1612.xlsx')
df_agro2 = pd.read_excel('tabela1613.xlsx')
#merging inputs into a single table
df_agro = pd.merge(df_agro1, df_agro2, on=['Cód.','Município'], how='inner')
#fill empty cells, some cities are present only in one of the tables
df_agro = df_agro.fillna(0)
#compute total production area
df_agro['Total prod'] = (df_agro['Produto das lavouras temporárias'].astype(int) + df_agro['Produto das lavouras permanentes'].astype(int))/100.
#keeping only the needed columns
df_agro_clean = df_agro[['Cód.','Total prod']]
#renaming columns
df_agro_clean.rename(columns = {'Cód.':'CityID', 'Total prod':'FertileArea'},
                              inplace = True)

df_agro_clean['CityID'] = df_agro_clean['CityID'].astype(str).str[:6]

df_agro_clean.head()

/home/rodrigolink/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-16-5812173770bb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agro_clean['CityID'] = df_agro_clean['CityID'].astype(str).str[:6]


,CityID,FertileArea
0,110001,177.87
1,110002,118.01
2,110003,469.47
3,110004,153.95
4,110005,674.26


In [17]:
len(df_agro_clean)

5534

Again, we have almost all cities in our data.

### 2.8. Wikipedia

Wikipedia is home to a ton of different information, but we have to dig a little to find something that will be useful. We will process two lists to help populate our data set.

In any apocalyptic scenario, the best way to guarantee our safety is to have a good collection of firearms and ammunition. We couldn't find the information about possesion of guns by city, so we will use the location and quantity of military bases as interesting places to get guns. There aren't that many around the country, so we will need to process this data afterwards.

In [18]:
#setting up the website 
url = 'https://en.wikipedia.org/wiki/List_of_Brazilian_military_bases'
#getting the data
html_data = requests.get(url).text
#processing the html data
soup = BeautifulSoup(html_data,"html5lib") 
#extracting the table from the html data
table_contents = []
tables = soup.findAll('table')
for table in tables:
    for row in table.findAll('tr'):
        temp=[]
        for cell in row.findAll('td'):
            temp.append(cell.text)
        table_contents.append(temp)
#creating dataframe and naming columns
df_military = pd.DataFrame(table_contents,columns=['Location', 'State', 'ICAO', 'Code', 'Basename','outro1','outro2'])
#droping first empty row
df_military = df_military.drop([0]).reset_index(inplace=False)
#keeping only the needed columns
df_military_clean = df_military[['Location','State']]
#replacing State with stateID
df_military_clean['State'] = df_military_clean['State'].replace(['Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', 'Ceará',
       'Federal District', 'Espírito Santo', 'Goiás', 'Maranhão', 'Mato Grosso', 'Mato Grosso do Sul', 
        'Minas Gerais', 'Pará', 'Paraíba', 'Paraná', 'Pernambuco', 'Piauí', 'Rio de Janeiro',
       'Rio Grande do Norte', 'Rio Grande do Sul', 'Rondônia', 'Roraima', 'Santa Catarina', 'São Paulo', 
        'Sergipe', 'Tocantins'],['AC', 'AL', 'AP', 'AM', 'BA', 'CE',
       'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 
        'RO', 'RR', 'SC', 'SP', 'SE', 'TO'])
#turning the name and state into a single column, in lowercase
df_military_clean['citystate'] = (df_military_clean['Location']+'-'+df_military_clean['State']).str.lower()
#getting the cityID from the area dataset
df_areamilitary = pd.merge(df_military_clean, df_area_clean, on='citystate', how='inner')
#counting the number of military bases by city
df_military_final = pd.DataFrame(df_areamilitary[['CityID']].value_counts(),columns=['MilitaryBases'])

df_military_final.reset_index(inplace=True)
df_military_final.head()

<ipython-input-18-df86a941ee1b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_military_clean['State'] = df_military_clean['State'].replace(['Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', 'Ceará',
<ipython-input-18-df86a941ee1b>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_military_clean['citystate'] = (df_military_clean['Location']+'-'+df_military_clean['State']).str.lower()


,CityID,MilitaryBases
0,330455,12
1,130260,3
2,240810,3
3,150140,2
4,530010,1


In [19]:
len(df_military_final)

20

The second list we will process is the number of airports available around the country, even the smaller local ones. It might not be easy to fly a light aircraft, but it is certainly a nice way out if you are in a hurry. There aren't that many around the country, so we will need to process this data afterwards.

In [20]:
#setting up the website 
url = 'https://en.wikipedia.org/wiki/List_of_airports_in_Brazil'
#getting the data
html_data = requests.get(url).text
#processing the html data
soup = BeautifulSoup(html_data,"html5lib") 
#extracting the table from the html data
table_contents = []
table = soup.findAll('table')
for row in table[0].findAll('tr'):
    temp=[]
    for cell in row.findAll('td'):
        temp.append(cell.text)
    table_contents.append(temp)

#creating dataframe and naming columns
df_airports = pd.DataFrame(table_contents,columns=['City','State','Sigla','Sigla2','Nome','outro'])
#droping first empty row
df_airports = df_airports.drop([0]).reset_index(inplace=False)
#keeping only the needed columns
df_airports_clean = df_airports[['City','State']]
#replacing State with stateID
df_airports_clean['State'] = df_airports_clean['State'].replace(['Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', 'Ceará',
       'Federal District', 'Espírito Santo', 'Goiás', 'Maranhão', 'Mato Grosso', 'Mato Grosso do Sul', 
        'Minas Gerais', 'Pará', 'Paraíba', 'Paraná', 'Pernambuco', 'Piauí', 'Rio de Janeiro',
       'Rio Grande do Norte', 'Rio Grande do Sul', 'Rondônia', 'Roraima', 'Santa Catarina', 'São Paulo', 
        'Sergipe', 'Tocantins'],['AC', 'AL', 'AP', 'AM', 'BA', 'CE',
       'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 
        'RO', 'RR', 'SC', 'SP', 'SE', 'TO'])

#turning the name and state into a single column, in lowercase
df_airports_clean['citystate'] = (df_airports_clean['City']+'-'+df_airports_clean['State']).str.lower()
#getting the cityID from the area dataset
df_areaairport = pd.merge(df_airports_clean, df_area_clean, on='citystate', how='inner')
#counting the number of airports by city
df_airport_final = pd.DataFrame(df_areaairport[['CityID']].value_counts(),columns=['Airports'])

df_airport_final.reset_index(inplace=True)
df_airport_final.head()

<ipython-input-20-02cb9c38a2c5>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_airports_clean['State'] = df_airports_clean['State'].replace(['Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', 'Ceará',
<ipython-input-20-02cb9c38a2c5>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_airports_clean['citystate'] = (df_airports_clean['City']+'-'+df_airports_clean['State']).str.lower()


,CityID,Airports
0,330455,3
1,150140,2
2,355030,2
3,310620,2
4,520870,2


In [21]:
len(df_airport_final)

233

### 2.9. Higher Education Census - IBGE

Every year IBGE releases a <a href="https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/censo-da-educacao-superior">census</a> with the information from all entities registered as Higher Education at the Ministry of Education.

Unless we are fortunate enough to have one of those hero types in our surviving group, we will need to learn new trades to have any chance of survival. We are considering any University or College as a source of useful information, as well as good engineering equipment.

In [22]:
#loading the file
df_universities = pd.read_csv('SUP_IES_2019.CSV',encoding='iso-8859-1' ,sep='|')
#counting the number of instituions by city
df_universities_final = pd.DataFrame(df_universities[['CO_MUNICIPIO']].value_counts(),columns=['Universities'])
df_universities_final.reset_index(inplace=True)
#renaming columns
df_universities_final.rename(columns = {'CO_MUNICIPIO':'CityID'},
                              inplace = True)
df_universities_final['CityID'] = df_universities_final['CityID'].astype(str).str[:6]
df_universities_final.head()

,CityID,Universities
0,355030,160
1,530010,72
2,330455,69
3,410690,58
4,310620,54


In [23]:
len(df_universities_final)

730

### 2.10. Public Libraries National System

The Special Secretary of Culture has a <a href="http://snbp.cultura.gov.br/">website</a> that gathers the data from the State Secretaries and all the public libraries. Again, we consider them a source of useful information that might prove vital to our survival. In this case, we have to process one file from each state.

In [40]:
#folder with all the state files
path='libraries/'
#creating an empty dataframe
df_libraries = pd.DataFrame(columns=['MUNICÍPIO','NOME DA BIBLIOTECA','VÍNCULO','ENDEREÇO', 'BAIRRO','TELEFONE','E-MAIL','outro','estado'])
#for each state
for filename in os.listdir(path):
        #read the file
        df_temp = pd.read_csv(path+filename,skiprows=3)
        #get stateID from filename
        df_temp['estado'] = filename[-6:-4]
        #add state dataframe to main dataframe
        df_libraries = df_libraries.append(df_temp,ignore_index=True)

#we need to fill the empty cells from 'MUNICÍPIO' with previous entry
df_libraries['MUNICÍPIO'] = df_libraries['MUNICÍPIO'].fillna(method='ffill')
#keeping only needed columns
df_libraries_clean = df_libraries[['MUNICÍPIO','estado']]
#turning the name and state into a single column, in lowercase
df_libraries_clean['citystate'] = (df_libraries_clean['MUNICÍPIO']+'-'+df_libraries_clean['estado']).str.lower()
#getting the cityID from the area dataset
df_arealibrary = pd.merge(df_libraries_clean, df_area_clean, on='citystate', how='inner')
df_libraries_final = pd.DataFrame(df_arealibrary[['CityID']].value_counts(),columns=['Libraries'])
df_libraries_final.reset_index(inplace=True)
df_libraries_final['CityID'] = df_libraries_final['CityID'].astype(str)
df_libraries_final.head()

<ipython-input-40-f1ccb27517c1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_libraries_clean['citystate'] = (df_libraries_clean['MUNICÍPIO']+'-'+df_libraries_clean['estado']).str.lower()


,CityID,Libraries
0,330455,103
1,355030,45
2,410690,24
3,310620,23
4,431490,23


In [25]:
len(df_libraries_final)

5067

### 2.11. Basedosdados.org

This website is a great resource for anybody working with Data Science. They provide links to different data sets and also process unstructured files into usable files. One of them is the <a href="https://basedosdados.org/dataset/matriz-de-vizinhanca-municipios-brasileiros">neighboring city matrix</a>. It is a two-column file, with two city IDs in each row, stating they are next to each other.

In [26]:
#loading file
df_neighbors = pd.read_csv('adjacency_matrix.csv')
#renaming columns
df_neighbors.rename(columns = {'id_munic_1':'CityID_1','id_munic_2':'CityID_2'},
                              inplace = True)

df_neighbors['CityID_1'] = df_neighbors['CityID_1'].astype(str).str[:6]
df_neighbors['CityID_2'] = df_neighbors['CityID_2'].astype(str).str[:6]

df_neighbors.head()

,CityID_1,CityID_2
0,170720,170025
1,170190,170025
2,170310,170025
3,170390,170025
4,170710,170025


### 2.12. GeoDataBR

GeoDataBR is a <a href="https://github.com/tbrugz/geodata-br">GitHub project</a> offering *geojson* files with the shape of the Brazilian cities.

In [27]:
city_json = 'geojs-100-mun.json'

Let's present this file creating a map showcasing the Health Human Development Index distribution.

In [ ]:
map_brazil = folium.Map(location = [-15.793889, -47.882778],
                       zoom_start=4)
folium.Choropleth(
    geo_data=city_json,
    name='health',
    data=df_atlas_clean,
    columns=['CityID_7', 'Health_Index'],
    key_on='properties.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='HealthIndex'
).add_to(map_brazil)
map_brazil

## 3. Cleaning the data

Now we will create a single dataframe gathering all the data. We will use the *CityID* as the key to join all the dataframes.

In [41]:
df_full = df_atlas_clean.merge(df_area_clean, on='CityID', how='outer')
df_full = df_full.merge(df_hospital, on='CityID', how='outer')
df_full = df_full.merge(df_energy_pivot, on='CityID', how='outer')
df_full = df_full.merge(df_solar_final, on='CityID', how='outer')
df_full = df_full.merge(df_climate_clean, on='CityID', how='outer')
df_full = df_full.merge(df_agro_clean, on='CityID', how='outer')
df_full = df_full.merge(df_military_final, on='CityID', how='outer')
df_full = df_full.merge(df_airport_final, on='CityID', how='outer')
df_full = df_full.merge(df_universities_final, on='CityID', how='outer')
df_full = df_full.merge(df_libraries_final, on='CityID', how='outer')
df_full.head()

,CityID,CityID_7,Population,Health_Index,City,State,Area,citystate,Hospital_beds,BiomassPower,...,SolarIncidence,Altitude,MeanTemperature,StdDevTemperature,MeanRainfall,FertileArea,MilitaryBases,Airports,Universities,Libraries
0,110001,1100015,24392.0,0.763,ALTA FLORESTA D'OESTE,RO,7067.127,alta floresta d'oeste-ro,49.0,NaN,...,3387.0,246.084366,25.416845,1.537877,167.144504,177.87,NaN,NaN,NaN,1.0
1,110002,1100023,90353.0,0.806,ARIQUEMES,RO,4426.571,ariquemes-ro,240.0,0.0,...,3067.0,165.899734,25.934812,1.225573,177.045622,118.01,NaN,NaN,3.0,1.0
2,110003,1100031,6313.0,0.757,CABIXI,RO,1314.352,cabixi-ro,9.0,NaN,...,3831.0,224.183029,25.554969,1.655509,162.875902,469.47,NaN,NaN,NaN,1.0
3,110004,1100049,78574.0,0.821,CACOAL,RO,3793.000,cacoal-ro,376.0,NaN,...,3374.0,229.949814,25.830520,1.342803,171.251407,153.95,NaN,1.0,3.0,1.0
4,110005,1100056,17029.0,0.799,CEREJEIRAS,RO,2783.300,cerejeiras-ro,40.0,NaN,...,3744.0,192.215805,25.640308,1.666064,165.300759,674.26,NaN,NaN,NaN,1.0


We have some cleaning to do. There are some empty cells that will be replaced by zero, but there are some that are needed for our project.

In [46]:
df_full.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CityID,5577,5572,330260,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CityID_7,5569,5565,3302007,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Population,5569,NaN,NaN,NaN,35444.7,219829,805,5236,10940,23726,1.12535e+07
Health_Index,5569,NaN,NaN,NaN,0.80159,0.0446767,0.672,0.769,0.808,0.836,0.894
City,5576,5298,SÃO DOMINGOS,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,5576,27,MG,853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Area,5576,NaN,NaN,NaN,1526.72,5603.28,3.565,204.695,417.548,1026.42,159533
citystate,5576,5572,rio de janeiro-rj,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hospital_beds,3553,NaN,NaN,NaN,131.202,763.344,1,20,35,70,29418
BiomassPower,1627,NaN,NaN,NaN,9589.73,33598.1,0,0,0,537,714783


Not really sure why, but here seems to be duplicates. We can see '*rio de janeiro-rj*' has two occurrences.

In [48]:
df_full = df_full.drop_duplicates()
df_full.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CityID,5573,5572,430000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CityID_7,5565,5565,1718451,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Population,5565,NaN,NaN,NaN,34277.8,203113,805,5235,10934,23424,1.12535e+07
Health_Index,5565,NaN,NaN,NaN,0.801564,0.0446809,0.672,0.769,0.808,0.836,0.894
City,5572,5298,BOM JESUS,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,5572,27,MG,853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Area,5572,NaN,NaN,NaN,1527.34,5605.24,3.565,204.64,417.548,1026.42,159533
citystate,5572,5572,são francisco de paula-rs,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hospital_beds,3549,NaN,NaN,NaN,126.797,718.108,1,20,35,69,29418
BiomassPower,1625,NaN,NaN,NaN,9599.6,33617.5,0,0,0,424,714783


There still are two occurrences for the *CityID* 430000. Let's check it.

In [49]:
df_full[df_full['CityID']=="430000"]

,CityID,CityID_7,Population,Health_Index,City,State,Area,citystate,Hospital_beds,BiomassPower,...,SolarIncidence,Altitude,MeanTemperature,StdDevTemperature,MeanRainfall,FertileArea,MilitaryBases,Airports,Universities,Libraries
5572,430000,NaN,NaN,NaN,LAGOA MIRIM,RS,2872.364,lagoa mirim-rs,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5573,430000,NaN,NaN,NaN,LAGOA DOS PATOS,RS,10196.942,lagoa dos patos-rs,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


These are the two largest lakes in Brazil that are in the Area dataset so that the sum of all the areas is equal to the country size. Let's drop them. 

In [50]:
df_full = df_full[df_full['CityID']!="430000"]

In [51]:
len(df_full)

5571

We need the 'CityID_7' column to be able to use the map feature. So let's drop all rows that are empty in this column.

In [52]:
df_full = df_full.dropna(subset=['CityID_7'])

In [53]:
len(df_full)

5565

We don't need 'citystate' anymore, so let's drop it.

In [54]:
df_full = df_full.drop(['citystate'], axis = 1)

In [55]:
df_full.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CityID,5565,5565,510700,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CityID_7,5565,5565,1718451,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Population,5565,NaN,NaN,NaN,34277.8,203113,805,5235,10934,23424,1.12535e+07
Health_Index,5565,NaN,NaN,NaN,0.801564,0.0446809,0.672,0.769,0.808,0.836,0.894
City,5565,5292,BOM JESUS,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,5565,27,MG,853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Area,5565,NaN,NaN,NaN,1525.06,5607.04,3.565,204.705,417.337,1024.64,159533
Hospital_beds,3549,NaN,NaN,NaN,126.797,718.108,1,20,35,69,29418
BiomassPower,1625,NaN,NaN,NaN,9599.6,33617.5,0,0,0,424,714783
WindPower,1625,NaN,NaN,NaN,11262.1,69508.1,0,0,0,0,1.17530e+06


Most of our data cover more than 5500 cities. For the other ones, we will create new columns using the distance to the cities that have that item. For example, we have only 20 cities with Military Bases. For one of the others, we will measure the distance to these 20 cities and use the sum of the inverse of the squared distance as a score. Those cities closer will have a higher score, while those far away will have a value close to zero.

We will do this for *HospitalBeds*, the energy columns, *MilitaryBases*, *Airports*, *Universities* and *Libraries*. These things aren't essential, but it is better to stay close to them.

To be able to do this, we need the coordinates for all cities. So we will keep only the 5538 cities that we have Latitude and Longitude.

In [58]:
df_full = df_full.dropna(subset=['Latitude'])
df_full.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CityID,5538,5538,510700,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CityID_7,5538,5538,1718451,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Population,5538,NaN,NaN,NaN,34148.3,202759,805,5238.25,10965,23768.2,1.12535e+07
Health_Index,5538,NaN,NaN,NaN,0.801618,0.04469,0.672,0.769,0.808,0.836,0.894
City,5538,5267,BOM JESUS,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,5538,27,MG,850,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Area,5538,NaN,NaN,NaN,1529.4,5620.03,3.565,204.749,418.048,1026.5,159533
Hospital_beds,3536,NaN,NaN,NaN,125.3,710.85,1,20,35,70,29418
BiomassPower,1620,NaN,NaN,NaN,9586.44,33633.7,0,0,0,480.5,714783
WindPower,1620,NaN,NaN,NaN,11296.8,69612.6,0,0,0,0,1.17530e+06


Just for completeness, let's also drop the one city we don't have *Altitude*.

In [59]:
df_full = df_full.dropna(subset=['Altitude'])
df_full.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CityID,5537,5537,510700,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CityID_7,5537,5537,1718451,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Population,5537,NaN,NaN,NaN,34153,202777,805,5238,10969,23769,1.12535e+07
Health_Index,5537,NaN,NaN,NaN,0.801622,0.0446931,0.672,0.769,0.808,0.836,0.894
City,5537,5266,BOM JESUS,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,5537,27,MG,850,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Area,5537,NaN,NaN,NaN,1529.61,5620.52,3.565,204.705,418.069,1026.54,159533
Hospital_beds,3536,NaN,NaN,NaN,125.3,710.85,1,20,35,70,29418
BiomassPower,1620,NaN,NaN,NaN,9586.44,33633.7,0,0,0,480.5,714783
WindPower,1620,NaN,NaN,NaN,11296.8,69612.6,0,0,0,0,1.17530e+06


Now all empty cells must be equal to zero.

In [62]:
df_full = df_full.fillna(0)
df_full.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
CityID,5537,5537,510700,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CityID_7,5537,5537,1718451,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Population,5537,NaN,NaN,NaN,34153,202777,805,5238,10969,23769,1.12535e+07
Health_Index,5537,NaN,NaN,NaN,0.801622,0.0446931,0.672,0.769,0.808,0.836,0.894
City,5537,5266,BOM JESUS,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,5537,27,MG,850,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Area,5537,NaN,NaN,NaN,1529.61,5620.52,3.565,204.705,418.069,1026.54,159533
Hospital_beds,5537,NaN,NaN,NaN,80.0181,571.216,0,0,18,47,29418
BiomassPower,5537,NaN,NaN,NaN,2804.77,18704.3,0,0,0,0,714783
WindPower,5537,NaN,NaN,NaN,3305.2,37994.8,0,0,0,0,1.17530e+06


It is a good time to back up this dataframe!

In [64]:
df_full.to_csv('df_full.csv',index=False)
df_full.head().T

,0,1,2,3,4
CityID,110001,110002,110003,110004,110005
CityID_7,1100015,1100023,1100031,1100049,1100056
Population,24392,90353,6313,78574,17029
Health_Index,0.763,0.806,0.757,0.821,0.799
City,ALTA FLORESTA D'OESTE,ARIQUEMES,CABIXI,CACOAL,CEREJEIRAS
State,RO,RO,RO,RO,RO
Area,7067.13,4426.57,1314.35,3793,2783.3
Hospital_beds,49,240,9,376,40
BiomassPower,0,0,0,0,0
WindPower,0,0,0,0,0
